# Deep dive into how gradients calculation works for hamiltonan neural network

### First, we will use `H(q,p) = p^2 + q^2` equation of mass-spring experiment and will calculate gradients using pytorch


In [1]:
# lets load the dataset first

from data import get_dataset

data = get_dataset(test_split=0.8)

train_x, train_dx = data['x'], data['dx']
test_x, test_dx = data['test_x'], data['test_dx']

In [2]:
sample_input_pair = train_x[0]
sample_output_pair = train_dx[0]
    

print("(p, q) pair is   - ", sample_input_pair)
print("(dp, dq) pair is - ", sample_output_pair)

(p, q) pair is   -  [-0.08470211  0.8053172 ]
(dp, dq) pair is -  [ 1.25313759 -0.28426143]


you can calculate this value by putting p, q value in hamiltonian function 

$$
H(q, p) = q^2 + p^2
$$


$p^2$ -> $-0.08470211 ^ 2$ = 0.00717444743  
$q^2$ -> $0.8053172 ^ 2$ = 0.64853579261

answer -> $p^2$ + $q^2$ = 0.65571024


In [3]:


from data import hamiltonian_fn

H = hamiltonian_fn(sample_input_pair)
H

array([0.65571024])

### Now lets do this with pytorch and simple MLP

In [16]:

import torch
z = torch.tensor(sample_input_pair, dtype=torch.float32, requires_grad=True)
y = torch.tensor(sample_output_pair, dtype=torch.float32)


In [17]:
from mlp import MLP

neural_net = MLP()

In [18]:
# get scalar value after forward pass
H = neural_net(z).squeeze(-1)
H

tensor(0.2264, grad_fn=<SqueezeBackward1>)

In [19]:
from torch.autograd import grad

grad_H = grad(H, z, create_graph=True)[0]
grad_H

tensor([0.0674, 0.0337], grad_fn=<ViewBackward0>)

In [20]:
import torch.nn as nn 

loss_fn = nn.MSELoss()
current_loss = loss_fn(grad_H, y)
current_loss


tensor(0.7535, grad_fn=<MseLossBackward0>)